# Convert NDJSON to Parquet Files

This notebook processes NDJSON files using PySpark, similar to the `convert_to_parquet` command.

## Setup

1. Upload NDJSON files to Databricks (DBFS or mount point)
2. Update the file paths in the configuration cell below
3. Run all cells to process files and generate Parquet files (5000 records per file)


In [ ]:
# Configuration
# Update these paths to point to your NDJSON files in Databricks

# Input: Path to NDJSON files (DBFS path or mounted path)
# Example: "/FileStore/ndjson_files/" or "/mnt/databricks/ndjson/"
NDJSON_INPUT_PATH = "/FileStore/ndjson_files/"

# Output: Path where Parquet files will be saved
# Example: "/FileStore/output_parquet/" or "/mnt/databricks/output/"
PARQUET_OUTPUT_PATH = "/FileStore/output_parquet/"

# Option 1: Process all .ndjson files in the input directory
PROCESS_ALL_FILES = True

# Option 2: Process specific files (if PROCESS_ALL_FILES is False)
SPECIFIC_FILES = [
    # "/FileStore/ndjson_files/file1.ndjson",
    # "/FileStore/ndjson_files/file2.ndjson",
]

# Schema inference: use samplingRatio to control how much of the file is read for schema inference
# 1.0 = read entire file (accurate but slow for large files)
# 0.1 = read 10% of file (faster but may miss some fields)
SCHEMA_SAMPLING_RATIO = 0.1  # Use 0.1 for large files, 1.0 for small files (<100MB)

# Number of rows per Parquet file/partition
ROWS_PER_PARTITION = 5000

print(f"Configuration:")
print(f"  Input path: {NDJSON_INPUT_PATH}")
print(f"  Output path: {PARQUET_OUTPUT_PATH}")
print(f"  Process all files: {PROCESS_ALL_FILES}")
print(f"  Schema sampling ratio: {SCHEMA_SAMPLING_RATIO}")
print(f"  Rows per Parquet partition: {ROWS_PER_PARTITION:,}")


In [ ]:
# Import required libraries
from pyspark.sql import SparkSession, functions as F
from pyspark.sql.types import StructType, ArrayType
from pyspark.sql.window import Window
import json
from pathlib import Path

# Get or create SparkSession (Databricks already has one)
spark = SparkSession.getActiveSession()
if spark is None:
    spark = SparkSession.builder.appName("NDJSONConverter").getOrCreate()

print("SparkSession initialized")
print(f"Spark version: {spark.version}")


In [ ]:
# Helper function: Auto-flatten DataFrame
def auto_flatten(df):
    """
    Recursively explode array<struct> columns and flatten struct columns into top-level columns.
    
    Column naming: Uses only the leaf key name, appending _1, _2, ... if names collide.
    """
    iteration = 0
    while True:
        iteration += 1
        schema = df.schema
        
        struct_cols = [
            f for f in schema.fields
            if isinstance(f.dataType, StructType)
        ]
        
        array_struct_cols = [
            f for f in schema.fields
            if isinstance(f.dataType, ArrayType)
            and isinstance(f.dataType.elementType, StructType)
        ]
        
        if not struct_cols and not array_struct_cols:
            # Nothing left to flatten/explode
            break
        
        print(f"Flatten iteration {iteration}: {len(struct_cols)} struct cols, {len(array_struct_cols)} array<struct> cols")
        
        # 1) Explode all array<struct> columns
        for f in array_struct_cols:
            print(f"  Exploding array<struct> column: {f.name}")
            df = df.withColumn(f.name, F.explode_outer(F.col(f.name)))
        
        # 2) Flatten all struct columns
        if struct_cols:
            struct_names = [f.name for f in struct_cols]
            print(f"  Flattening struct columns: {struct_names}")
            
            # Track used names so we don't collide
            used_names = set(df.columns)
            new_cols = []
            
            for f in df.schema.fields:
                if isinstance(f.dataType, StructType):
                    # Replace struct with its fields
                    for subf in f.dataType.fields:
                        base_name = subf.name  # Only last-level key
                        new_name = base_name
                        
                        # Avoid collisions by appending _1, _2, ...
                        i = 1
                        while new_name in used_names:
                            new_name = f"{base_name}_{i}"
                            i += 1
                        
                        used_names.add(new_name)
                        new_cols.append(F.col(f"{f.name}.{subf.name}").alias(new_name))
                else:
                    # Keep non-struct column as-is
                    new_cols.append(F.col(f.name))
            
            df = df.select(new_cols)
    
    print(f"Flattening complete after {iteration} iteration(s)")
    return df

print("auto_flatten function defined")


In [ ]:
# Helper function: Restructure negotiated rate DataFrame
def restructure_negotiated_rate_df(df):
    """
    Convert mixed negotiated-rate dataset into a unified dataframe.
    
    Steps:
    1. Split billing rows and provider rows
    2. Join billing rows to provider rows based on provider_references array
    3. Aggregate all NPIs from all matching providers into a single all_npis column
    4. Keep one row per billing record (no explosion of provider_references)
    """
    # Check if required columns exist
    if "provider_group_id" not in df.columns or "provider_references" not in df.columns:
        print("Missing required columns for restructure_negotiated_rate_df. Skipping restructuring.")
        return df
    
    # 1. Identify billing rows (billing_code not null)
    billing_df = df.filter(F.col("billing_code").isNotNull())
    
    # 2. Identify provider metadata rows (provider_group_id not null)
    provider_df = df.filter(F.col("provider_group_id").isNotNull())
    
    # 3. Select only provider-specific columns from provider_df
    billing_cols_set = set(billing_df.columns)
    provider_cols_to_keep = ["provider_group_id"]
    
    # Rename npi to provider_npi to avoid ambiguity
    if "npi" in provider_df.columns:
        provider_df = provider_df.withColumnRenamed("npi", "provider_npi")
        provider_cols_to_keep.append("provider_npi")
    
    # Add any other columns from provider_df that don't exist in billing_df
    for col_name in provider_df.columns:
        if col_name not in billing_cols_set and col_name not in provider_cols_to_keep and col_name != "provider_group_id":
            provider_cols_to_keep.append(col_name)
    
    # Select only needed columns from provider_df
    provider_df_clean = provider_df.select(*provider_cols_to_keep)
    
    # 4. Temporarily explode provider_references to join, then aggregate back
    billing_df_with_id = billing_df.withColumn(
        "_billing_row_id", 
        F.monotonically_increasing_id()
    )
    
    # Explode provider_references for the join
    billing_exploded = billing_df_with_id.withColumn(
        "provider_reference", 
        F.explode_outer("provider_references")
    )
    
    # 5. Join on provider_reference == provider_group_id
    provider_df_for_join = provider_df_clean.withColumnRenamed("provider_group_id", "_join_provider_group_id")
    
    joined = billing_exploded.join(
        provider_df_for_join,
        on=[billing_exploded["provider_reference"] == provider_df_for_join["_join_provider_group_id"]],
        how="left"
    )
    
    # Drop temporary columns used for join
    joined = joined.drop("provider_reference", "_join_provider_group_id")
    
    # 6. Group back by _billing_row_id and aggregate all NPIs
    aggregated = joined.groupBy("_billing_row_id").agg(
        # Take first value of each billing column
        *[F.first(col).alias(col) for col in billing_df.columns if col != "provider_references"],
        # Keep provider_references as-is
        F.first("provider_references").alias("provider_references"),
        # Collect all NPIs from all matched providers into a single flattened array
        F.array_distinct(
            F.flatten(
                F.collect_list(F.coalesce(F.col("provider_npi"), F.array()))
            )
        ).alias("all_npis")
    ).drop("_billing_row_id")
    
    return aggregated

print("restructure_negotiated_rate_df function defined")


In [ ]:
# Helper function: Process a single NDJSON file
def process_ndjson_file(file_path, output_path, schema_sampling_ratio=0.1, rows_per_partition=5000):
    """
    Process a single NDJSON file:
    1. Infer schema (with sampling)
    2. Read file with schema
    3. Apply auto_flatten
    4. Apply restructure_negotiated_rate_df if applicable
    5. Write to Parquet files (with specified rows per partition)
    """
    print(f"\n{'='*60}")
    print(f"Processing: {file_path}")
    print(f"{'='*60}")
    
    try:
        # Step 1: Infer schema from file (with sampling)
        print(f"Step 1: Inferring schema (sampling ratio: {schema_sampling_ratio})...")
        df_schema = spark.read.option("samplingRatio", str(schema_sampling_ratio)).json(file_path)
        schema = df_schema.schema
        print(f"  Inferred schema with {len(schema.fields)} fields")
        
        # Step 2: Read file with inferred schema
        print("Step 2: Reading NDJSON file with schema...")
        df = spark.read.schema(schema).json(file_path)
        initial_row_count = df.count()
        print(f"  Initial row count: {initial_row_count:,}")
        
        # Step 3: Apply auto_flatten
        print("Step 3: Flattening nested structures...")
        df = auto_flatten(df)
        flattened_row_count = df.count()
        print(f"  Flattened row count: {flattened_row_count:,}")
        print(f"  Columns after flattening: {len(df.columns)}")
        
        # Step 4: Apply restructure_negotiated_rate_df if applicable
        print("Step 4: Checking if restructure_negotiated_rate_df is applicable...")
        if "provider_group_id" in df.columns and "provider_references" in df.columns:
            print("  Applying restructure_negotiated_rate_df...")
            df = restructure_negotiated_rate_df(df)
            restructured_row_count = df.count()
            print(f"  Restructured row count: {restructured_row_count:,}")
        else:
            print("  Skipping restructure_negotiated_rate_df (required columns not present)")
        
        # Step 5: Calculate number of partitions needed
        row_count = df.count()
        num_partitions = max(1, (row_count + rows_per_partition - 1) // rows_per_partition)
        print(f"\nStep 5: Writing to Parquet files...")
        print(f"  Total rows: {row_count:,}")
        print(f"  Rows per partition: {rows_per_partition:,}")
        print(f"  Number of partitions: {num_partitions}")
        
        # Step 6: Write to Parquet with repartitioning
        # Generate output directory name (based on input filename)
        file_name = Path(file_path).name.replace('.ndjson', '')
        parquet_output_dir = f"{output_path.rstrip('/')}/{file_name}/"
        
        # Repartition to get approximately rows_per_partition rows per partition
        # Use repartition (not coalesce) to allow both increasing and decreasing partitions
        df_repartitioned = df.repartition(num_partitions)
        
        # Write to Parquet
        df_repartitioned.write.mode("overwrite").parquet(parquet_output_dir)
        
        print(f"  Saved to: {parquet_output_dir}")
        print(f"  Total rows written: {row_count:,}")
        print(f"  Columns: {len(df.columns)}")
        
        # Show a preview of the data
        print("\nPreview of data (first 10 rows):")
        display(df.limit(10))
        
        return {
            "success": True,
            "file_path": file_path,
            "output_dir": parquet_output_dir,
            "total_rows": row_count,
            "partitions": num_partitions,
            "rows_per_partition": rows_per_partition,
            "columns": len(df.columns)
        }
        
    except Exception as e:
        print(f"ERROR processing {file_path}: {str(e)}")
        import traceback
        traceback.print_exc()
        return {
            "success": False,
            "file_path": file_path,
            "error": str(e)
        }

print("process_ndjson_file function defined")


In [ ]:
# Get list of files to process
import os

if PROCESS_ALL_FILES:
    # List all .ndjson files in the input directory
    print(f"Listing .ndjson files in: {NDJSON_INPUT_PATH}")
    files = []
    try:
        # Use dbutils to list files (Databricks utility)
        file_list = dbutils.fs.ls(NDJSON_INPUT_PATH)
        files = [f.path for f in file_list if f.name.endswith('.ndjson')]
        print(f"Found {len(files)} .ndjson file(s)")
    except Exception as e:
        print(f"Error listing files: {e}")
        print("Trying alternative method...")
        # Alternative: use Spark to list files
        try:
            files_df = spark.read.format("binaryFile").load(f"{NDJSON_INPUT_PATH}*.ndjson")
            files = [row.path for row in files_df.select("path").distinct().collect()]
            print(f"Found {len(files)} .ndjson file(s) using alternative method")
        except Exception as e2:
            print(f"Alternative method also failed: {e2}")
            files = []
else:
    # Use specific files
    files = SPECIFIC_FILES
    print(f"Processing {len(files)} specific file(s)")

if not files:
    print("ERROR: No files to process!")
else:
    print("\nFiles to process:")
    for i, f in enumerate(files, 1):
        print(f"  {i}. {f}")


In [ ]:
# Process all files
if not files:
    print("No files to process. Please check the configuration.")
else:
    # Create output directory if it doesn't exist
    try:
        dbutils.fs.mkdirs(PARQUET_OUTPUT_PATH)
        print(f"Output directory ready: {PARQUET_OUTPUT_PATH}")
    except:
        pass
    
    # Process each file
    results = []
    for file_path in files:
        result = process_ndjson_file(
            file_path=file_path,
            output_path=PARQUET_OUTPUT_PATH,
            schema_sampling_ratio=SCHEMA_SAMPLING_RATIO,
            rows_per_partition=ROWS_PER_PARTITION
        )
        results.append(result)
    
    # Summary
    print(f"\n{'='*60}")
    print("PROCESSING SUMMARY")
    print(f"{'='*60}")
    successful = [r for r in results if r.get("success")]
    failed = [r for r in results if not r.get("success")]
    
    print(f"Total files processed: {len(results)}")
    print(f"  Successful: {len(successful)}")
    print(f"  Failed: {len(failed)}")
    
    if successful:
        total_rows = sum(r.get("total_rows", 0) for r in successful)
        total_partitions = sum(r.get("partitions", 0) for r in successful)
        avg_columns = sum(r.get("columns", 0) for r in successful) / len(successful) if successful else 0
        print(f"\nSuccessful files:")
        print(f"  Total rows processed: {total_rows:,}")
        print(f"  Total Parquet partitions created: {total_partitions}")
        print(f"  Average columns per file: {avg_columns:.0f}")
        print(f"\nOutput directories:")
        for r in successful:
            print(f"  - {r['output_dir']}")
            print(f"    ({r['total_rows']:,} total rows in {r['partitions']} partition(s), ~{r['rows_per_partition']:,} rows per partition)")
    
    if failed:
        print(f"\nFailed files:")
        for r in failed:
            print(f"  - {r['file_path']}: {r.get('error', 'Unknown error')}")


## Notes

- **Schema Inference**: For large files, use `SCHEMA_SAMPLING_RATIO = 0.1` (10%) to speed up processing. For small files or if you need 100% accuracy, use `1.0` (read entire file).
- **Output Files**: Parquet files are saved to subdirectories (one per input file) in the output path. Each Parquet file contains approximately `ROWS_PER_PARTITION` (5000) rows.
- **Partitioning**: The data is repartitioned to create files with approximately the specified number of rows per partition. The actual number of partitions depends on the total row count.
- **Databricks Paths**: 
  - Use `/FileStore/` for files uploaded through Databricks UI
  - Use `/mnt/` for mounted cloud storage (S3, ADLS, etc.)
  - Use `dbfs:/` prefix for DBFS paths
- **Memory**: For very large files, consider processing one file at a time or increasing cluster size.
- **Reading Parquet Files**: You can read the output Parquet files using:
  ```python
  df = spark.read.parquet("/FileStore/output_parquet/filename/")
  ```
